In [ ]:
%load_ext autoreload
%autoreload 2
from copy import deepcopy
import os
os.chdir("/home/jakob/doktor/projects/EnsembleUncertainty/code")
"""Learing "logit" distribution in regression example"""
from pathlib import Path
from datetime import datetime
import logging
import numpy as np
import matplotlib.pyplot as plt
import torch

import src.dataloaders.uci.wine as uci_wine
import src.dataloaders.uci.bost as uci_bost
import src.utils as utils
from src.distilled import logits_probability_distribution
from src.ensemble import ensemble
from src.ensemble import sep_regressor, mean_regressor, simple_regressor
import src.metrics as metrics

LOGGER = logging.getLogger(__name__)
EXPERIMENT_NAME = "regression_logits"

# Settings
class Args():
    pass
args = Args()
args.seed = 1
args.gpu = False
args.log_dir = Path("./logs")
args.log_level = logging.INFO
args.retrain = True

args.num_ensemble_members=1
args.num_epochs=1
args.lr = 0.01

# General constructs
rmse = metrics.Metric(name="RMSE", function=metrics.root_mean_squared_error)
BATCH_SIZE = 32

In [ ]:
log_file = Path("{}_{}.log".format(
    EXPERIMENT_NAME,
    datetime.now().strftime("%Y%m%d_%H%M%S")))
utils.setup_logger(log_path=Path.cwd() / args.log_dir / log_file,
                   log_level=args.log_level)
LOGGER.info("Args: {}".format(args))
device = utils.torch_settings(args.seed, args.gpu)
LOGGER.info("Creating dataloader")
wine_training_data, wine_validation_data = uci_wine.WineData(
    "~/doktor/datasets/UCI/wine/winequality-red.csv").create_train_val_split(
    training_samples_ratio=0.9)

bost_training_data, bost_validation_data = uci_bost.BostonData(
    "/home/jakob/doktor/datasets/UCI/bost/housing.data").create_train_val_split(
    training_samples_ratio=0.9)

training_data, validation_data = wine_training_data, wine_validation_data
#training_data, validation_data = bost_training_data, bost_validation_data


input_size = training_data.input_dim
layer_sizes = [input_size, 50, 2]
ensemble_output_size = layer_sizes[-1]
args.num_ensemble_members = 1
args.num_epochs=40
args.lr = 0.01
args.log_level = logging.INFO
train_loader = torch.utils.data.DataLoader(training_data,
                                           batch_size=BATCH_SIZE,
                                           shuffle=True)
validation_loader = torch.utils.data.DataLoader(validation_data,
                                                batch_size=BATCH_SIZE,
                                                shuffle=True)

prob_ensemble = ensemble.Ensemble(ensemble_output_size)
for _ in range(args.num_ensemble_members):
    model = simple_regressor.SimpleRegressor(layer_sizes,
                                             device=device,
                                             variance_transform=utils.variance_linear_asymptote,
                                             learning_rate=args.lr)
    prob_ensemble.add_member(model)
prob_ensemble.add_metrics([rmse])

prob_ensemble.train(train_loader, args.num_epochs, validation_loader=validation_loader)

In [ ]:
new_lr /= 10
dir(model.optimizer)
model.optimizer.defaults["lr"] = new_lr
model.train(train_loader, args.num_epochs, validation_loader=validation_loader)

In [ ]:
model = prob_ensemble.members[0]
for param in model.layers[1].parameters():
    print(param.grad)

In [ ]:
log_file = Path("{}_{}.log".format(
    EXPERIMENT_NAME,
    datetime.now().strftime("%Y%m%d_%H%M%S")))
utils.setup_logger(log_path=Path.cwd() / args.log_dir / log_file,
                   log_level=args.log_level)
LOGGER.info("Args: {}".format(args))
device = utils.torch_settings(args.seed, args.gpu)
LOGGER.info("Creating dataloader")
wine_training_data, wine_validation_data = uci_wine.WineData(
    "~/doktor/datasets/UCI/wine/winequality-red.csv").create_train_val_split(
    training_samples_ratio=0.9)

bost_training_data, bost_validation_data = uci_bost.BostonData(
    "/home/jakob/doktor/datasets/UCI/bost/housing.data").create_train_val_split(
    training_samples_ratio=0.9)

training_data, validation_data = wine_training_data, wine_validation_data
#training_data, validation_data = bost_training_data, bost_validation_data


input_size = training_data.input_dim
layer_sizes = [input_size, 50, 1]
ensemble_output_size = layer_sizes[-1]
args.num_ensemble_members = 1
args.num_epochs=40
args.lr = 0.01
args.log_level = logging.INFO
train_loader = torch.utils.data.DataLoader(training_data,
                                           batch_size=BATCH_SIZE,
                                           shuffle=True)
validation_loader = torch.utils.data.DataLoader(validation_data,
                                                batch_size=BATCH_SIZE)

prob_ensemble = ensemble.Ensemble(ensemble_output_size)
for _ in range(args.num_ensemble_members):
    model = mean_regressor.MeanRegressor(layer_sizes,
                                             device=device,
                                             learning_rate=args.lr)
    prob_ensemble.add_member(model)
prob_ensemble.add_metrics([rmse])

loss = model.train(train_loader, args.num_epochs, validation_loader=validation_loader)


In [ ]:
train_loss, val_loss = np.array(loss["Train"]), np.array(loss["Validation"])
epochs = np.arange(1, args.num_epochs + 1)

fig, ax = plt.subplots()

ax.plot(epochs, train_loss)
ax.plot(epochs, val_loss)

In [ ]:
wine_training_data, wine_validation_data = uci_wine.WineData(
    "~/doktor/datasets/UCI/wine/winequality-red.csv").create_train_val_split(
    training_samples_ratio=0.9)